In [ ]:
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader
from DallEdVAE import dVAE
from train_dVAE import train
import numpy as np
import time
import pickle

# Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 256
epochs = 20
lr = 1e-3
inp_ch = 3
n_hid = 256
n_init = 128
bpg = 2
K = 8192
D = 512
Beta = 6.6
print_step = 10
path_to_data = None
path_to_model = None

model = dVAE(inp_ch, n_hid, n_init, bpg, K, D, Beta).to(device)

if path_to_model != None:
    model.load_state_dict(torch.load(path_to_model))

optimizer = optim.Adam(model.parameters(), lr=lr, amsgrad=False)

# Loading the data
with open(path_to_data, 'rb') as f:
    dataset = pickle.load(f)

class MyDataset(Dataset):
    def __init__(self, dataset):
      super().__init__()
      self.dataset = dataset
    def __len__(self):
      return len(self.dataset)
    def __getitem__(self, idx):
      img = self.dataset[idx]
      return img

img_dataset = MyDataset(dataset)
data_var = 1#np.var(img_dataset) #in bayad mohasebe beshe
dataloader = DataLoader(img_dataset, batch_size=batch_size, shuffle=True)

# Training
start_time = time.time()
total_L, rec_L, vq_L, perplexities = train(model, optimizer, dataloader, epochs, data_var,
                                           K, device, print_step, batch_size)
end_time = time.time()
print(f"Training time: {end_time- start_time:.3f} seconds")